In [1]:
import math
import pandas as pd
from itertools import product

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
input = [item.strip('\n') for item in open('test_input.txt').readlines()]
input

['467..114..',
 '...*......',
 '..35..633.',
 '......#...',
 '617*......',
 '.....+.58.',
 '..592.....',
 '......755.',
 '...$.*....',
 '.664.598..']

In [3]:
input = [item.strip('\n') for item in open('input_03.txt').readlines()]

In [4]:
def get_touched_coords(surr_coords, symbol_coords):
    coords = []

    for sym in symbol_coords:
        for sur in surr_coords:
            if (sym[1] == sur[0]) and (sym[2] == sur[1]):
                coords.append(sym)

    return coords

def get_surr_coords(coords, row_cnt, col_cnt):
    min_r = coords[0] - 1 if coords[0] - 1 >= 0 else 0
    max_r = coords[0] + 1 if coords[0] + 1 < row_cnt else row_cnt 
    min_c = coords[1] - 1 if coords[1] - 1 >= 0 else 0
    max_c = coords[1] + 1 if coords[1] + 1 < col_cnt else col_cnt 

    return list(product(range(min_r, max_r+1), range(min_c, max_c+1)))

In [5]:
symbol_coords = []

for row_index, row in enumerate(input):
    for col_index, c in enumerate(row):
        if c not in '0123456789.':
            symbol_coords.append((c, row_index, col_index))

In [6]:
row_cnt = len(input)
col_cnt = len(input[0])
dist = 1

number_coords = {}
res = 0
anal = []
curr_num = ''
curr_num_coords = []

for row_index, row in enumerate(input):
    for col_index, c in enumerate(row):
        if (not c.isnumeric() and len(curr_num) > 0) or (col_index == 0 and curr_num != ''):
            surround_coords = []

            for coords in curr_num_coords:
                surround_coords.extend(get_surr_coords(coords, row_cnt, col_cnt))

            surround_coords = [item for item in {(a, b): None for a, b in surround_coords}.keys() if item not in curr_num_coords]

            touch_list = get_touched_coords(surround_coords, symbol_coords)
            if len(touch_list) > 0:
                number_coords[int(curr_num)] = {
                    'number_coords': curr_num_coords,
                    'surround_coords': surround_coords,
                    }
                res += int(curr_num)
                anal.append([int(curr_num), touch_list, curr_num_coords, surround_coords])

            curr_num = ''
            curr_num_coords = []            
        elif c.isnumeric():
            curr_num += c
            curr_num_coords.append((row_index, col_index))

res

531932

In [7]:
# explode list of touples of characters with coords into columns and keep just chars
df_destilated_chars = pd.DataFrame(
    pd.DataFrame(
        pd.DataFrame(anal, columns=['num', 'touch_list', 'num_crds', 'surr_crds']).touch_list.to_list(),
        columns=['touch_list']
        ).touch_list.to_list(),
        columns=['char', 'r', 'c']
    ).loc[:, ['char']]

#join chars from above and filter relevant rows, then group by, multiply and filter for 2 neighbors
(pd
    .DataFrame(anal, columns=['num', 'touch_list', 'num_crds', 'surr_crds'])
    .join(df_destilated_chars)
    .query('char == "*"')
    .loc[:, ['num', 'touch_list']]
    .astype({'touch_list': str})
    .groupby('touch_list')
    .agg([math.prod, 'count'])
    .reset_index()
    .pipe(lambda df: df.loc[df[('num', 'count')] == 2, ('num', 'prod')])
    .sum()
    )

73646890